# DS5

In [41]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scipy
import geopy
import folium

## Prep

In [42]:
dtf = pd.read_csv("./directory.csv")
dtf = dtf[dtf["City"] == 'Las Vegas']
dtf = dtf[["City", 'Street Address', 'Longitude', 'Latitude']]
dtf.head()

,City,Street Address,Longitude,Latitude
20644,Las Vegas,4507 Flamingo Rd,-115.20,36.12
20645,Las Vegas,"475 E Windmill Lane, Fashion Show",-115.15,36.04
20646,Las Vegas,"3200 LAS VEGAS BLVD. S., STE 1795",-115.17,36.13
20647,Las Vegas,8350 W Cheyenne Ave,-115.28,36.22
20648,Las Vegas,3730 LAS VEGAS BLVD S,-115.18,36.11


## Der neun columns

In [43]:
dtf['Potential'] = np.random.randint(low=3, high=11, size=len(dtf))
dtf['Staff'] = dtf['Potential'].apply(lambda x: int(np.random.rand()*x)+1)
dtf['Capacity'] = dtf['Potential'] - dtf['Staff']
dtf["cost"] = np.random.choice (['high', 'medium', 'low'], size=len(dtf), p=[0.4, 0.5, 0.1])
dtf.head()

,City,Street Address,Longitude,Latitude,Potential,Staff,Capacity,cost
20644,Las Vegas,4507 Flamingo Rd,-115.20,36.12,4,1,3,medium
20645,Las Vegas,"475 E Windmill Lane, Fashion Show",-115.15,36.04,5,1,4,medium
20646,Las Vegas,"3200 LAS VEGAS BLVD. S., STE 1795",-115.17,36.13,10,10,0,high
20647,Las Vegas,8350 W Cheyenne Ave,-115.28,36.22,8,7,1,medium
20648,Las Vegas,3730 LAS VEGAS BLVD S,-115.18,36.11,8,2,6,medium


## Lat & Lnt

In [44]:
city = 'Las Vegas'
locator = geopy.geocoders.Nominatim(user_agent= 'MyCoder')
location = locator.geocode(city)
print(location)

Las Vegas, Clark County, Nevada, United States


In [45]:
location = [location.latitude, location.longitude]
print('[lat, long]: ', location)

[lat, long]:  [36.1672559, -115.148516]


In [46]:
colours = []
for i in dtf['cost']:
    if i=='high': colours.append('red')
    if i=='medium': colours.append('orange')
    if i=='low': colours.append('green')

dtf['Colour'] = colours
dtf.head()

,City,Street Address,Longitude,Latitude,Potential,Staff,Capacity,cost,Colour
20644,Las Vegas,4507 Flamingo Rd,-115.20,36.12,4,1,3,medium,orange
20645,Las Vegas,"475 E Windmill Lane, Fashion Show",-115.15,36.04,5,1,4,medium,orange
20646,Las Vegas,"3200 LAS VEGAS BLVD. S., STE 1795",-115.17,36.13,10,10,0,high,red
20647,Las Vegas,8350 W Cheyenne Ave,-115.28,36.22,8,7,1,medium,orange
20648,Las Vegas,3730 LAS VEGAS BLVD S,-115.18,36.11,8,2,6,medium,orange


## Map man

In [50]:
map_ = folium.Map(location=location, tiles='cartodb positron')
dtf.apply(lambda row: folium.CircleMarker(
           location=[row["Latitude"],row["Longitude"]], popup=row["Street Address"],
           color=row["Colour"], fill=True,
           radius=row["Staff"]).add_to(map_), axis=1)
map_